# What percent of talk pages have not yet been created?

[Task](https://phabricator.wikimedia.org/T272657)

# Background

This task is about uncovering the percentage of talk pages, across namespaces and Wikipedias, that have not yet been created. nowing the percentage of talk pages, across namespaces and Wikipedias, that have not yet been created will help us decide how highly we should prioritize work on designing the empty state experience (See [T252902](https://phabricator.wikimedia.org/T252902).


# Metrics

* We are curious to know the following: Of all the pages in the subject namespace, what percentage of them do NOT have a corresponding talk page that's been created?
* We would value seeing the percentage of non-yet-created talk pages grouped by Wikipedia and within each wiki, grouped by namespace.


In [71]:
shhh <- function(expr) suppressPackageStartupMessages(suppressWarnings(suppressMessages(expr)))
shhh({
    library(tidyverse);
})

In [84]:
test <- missing_talk_pages)

,wiki,subject_namespace,num_subject_pages,num_talk_pages,num_talk_pages_not_yet_created,prop_talk_pages_not_yet_created
,<chr>,<int>,<int>,<int>,<int>,<dbl>
1,aawiki,0,2,1,1,0.5000000
2,aawiki,2,170,54,116,0.6823529
3,aawiki,4,7,0,7,1.0000000
4,aawiki,8,107,0,107,1.0000000
5,aawiki,10,24,0,24,1.0000000
6,aawiki,14,29,0,29,1.0000000


In [87]:
query <-

"
--find all subject namespace page
WITH subject_pages AS (
SELECT
-- address duplicate page title instances in mediawiki_page_history
    DISTINCT page_title AS subject_title,
    wiki_db AS wiki,
     CASE
            WHEN page_namespace = 0  THEN 'Main/Article'
            WHEN page_namespace = 2  THEN 'User'
            WHEN page_namespace = 4  THEN 'Wikipedia'
            WHEN page_namespace = 6  THEN 'File'
            WHEN page_namespace = 8  THEN 'MediaWiki'
            WHEN page_namespace = 10  THEN 'Template'
            WHEN page_namespace = 12  THEN 'Help'
            WHEN page_namespace = 14  THEN 'Category'
            WHEN page_namespace = 100  THEN 'Portal'
            WHEN page_namespace = 118  THEN 'Draft'
            WHEN page_namespace = 710  THEN 'TimedText'
            WHEN page_namespace = 828  THEN 'Module'
            END AS subject_namespace
FROM wmf.mediawiki_page_history
INNER JOIN canonical_data.wikis
    ON
        wiki_db = database_code and
        database_group ==  'wikipedia'
WHERE 
-- review all primary subject namespaces
   page_namespace IN (0,2,4,6,8,10,12,14,100,118, 710, 828) 
--Remove redirects
  AND page_is_redirect = FALSE 
--Remove archived articles 
  AND page_is_deleted = FALSE
  AND snapshot = '2020-12'
),
--find all talk namespace pages
talk_pages AS (
SELECT
    DISTINCT page_title AS talk_title,
    wiki_db AS wiki,
    CASE
            WHEN page_namespace = 1  THEN 'Talk'
            WHEN page_namespace = 3  THEN 'User talk'
            WHEN page_namespace = 5  THEN 'Wikipedia talk'
            WHEN page_namespace = 7  THEN 'File talk'
            WHEN page_namespace = 9  THEN 'MediaWiki talk'
            WHEN page_namespace = 11  THEN 'Template talk'
            WHEN page_namespace = 13  THEN 'Help talk'
            WHEN page_namespace = 15  THEN 'Category talk'
            WHEN page_namespace = 101  THEN 'Portal talk'
            WHEN page_namespace = 119  THEN 'Draft talk'
            WHEN page_namespace = 711  THEN 'TimedText talk'
            WHEN page_namespace = 829  THEN 'Module talk'
            END AS talk_namespace
FROM wmf.mediawiki_page_history
INNER JOIN canonical_data.wikis
    ON
        wiki_db = database_code and
        database_group ==  'wikipedia'
WHERE 
-- review all primary talk namespaces
  page_namespace IN (1,3,5,7,9,11,13,15, 101, 119, 711, 829)  
--Remove redirects
  AND page_is_redirect = FALSE 
--Remove archived articles 
   AND page_is_deleted = FALSE
   AND snapshot = '2020-12'
)
-- MAIN QUERY --
SELECT
    sp.wiki,
    sp.subject_namespace,
    COUNT(*) AS num_subject_pages,
    SUM(CAST(talk_title IS NOT NULL AS int)) AS num_talk_pages_created,
    SUM(CAST(talk_title IS NULL AS int)) AS num_talk_pages_not_yet_created
 FROM 
    subject_pages AS sp
LEFT JOIN talk_pages ON 
    sp.subject_title = talk_pages.talk_title AND
    sp.wiki = talk_pages.wiki 
GROUP BY
    sp.wiki,
    sp.subject_namespace
    ;
"

In [88]:
missing_talk_pages <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



# By Namespace

In [91]:
missing_talk_bynamespace <- missing_talk_pages %>%
    group_by(subject_namespace) %>%
    summarise(total_talk_missing = sum(num_talk_pages_not_yet_created),
              total_subject = sum(num_subject_pages),
              prop_talk_missing = round(sum(num_talk_pages_not_yet_created)/sum(num_subject_pages) * 100,2), .groups = 'drop')

missing_talk_bynamespace

subject_namespace,total_talk_missing,total_subject,prop_talk_missing
<chr>,<int>,<int>,<dbl>
Category,9803368,13485972,72.69
Draft,50342,66563,75.63
File,2116032,2699583,78.38
Help,18252,22739,80.27
Main/Article,32239028,56107661,57.46
MediaWiki,106603,115030,92.67
Module,148977,157656,94.49
Portal,471198,523353,90.03
Template,4398181,5219632,84.26


# By Wiki and Talk Namespace

In [95]:
missing_talk_bynamespace <- missing_talk_pages %>%
    group_by(wiki, subject_namespace) %>%
    summarise(total_talk_missing = sum(num_talk_pages_not_yet_created),
              total_subject = sum(num_subject_pages),
              prop_talk_missing = round(sum(num_talk_pages_not_yet_created)/sum(num_subject_pages) * 100,2), .groups = 'drop')

missing_talk_bynamespace

wiki,subject_namespace,total_talk_missing,total_subject,prop_talk_missing
<chr>,<chr>,<int>,<int>,<dbl>
aawiki,Category,29,29,100.00
aawiki,Main/Article,1,2,50.00
aawiki,MediaWiki,107,107,100.00
aawiki,Template,24,24,100.00
aawiki,User,116,170,68.24
aawiki,Wikipedia,7,7,100.00
abwiki,Category,7046,7086,99.44
abwiki,File,10,10,100.00
abwiki,Help,2,2,100.00
